<a href="https://colab.research.google.com/github/teotavares2/RecPad/blob/main/Projeto_Rec_Pad_Digits_vFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digits vFinal

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import f1_score,recall_score, precision_score, confusion_matrix, roc_curve, classification_report, accuracy_score
from time import time,sleep
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import os
%matplotlib inline
import _pickle as cPickle
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:
# Classifiers

classifiers =[]
mlp = MLPClassifier(random_state=42, verbose=1, batch_size=64, max_iter=400,early_stopping = True)
classifiers.append(mlp)

knn = KNeighborsClassifier(n_jobs=-1) #algorithm='auto'
classifiers.append(knn)

svm = SVC(probability=True)
classifiers.append(svm)

# Classifiers Parameters
parameters=[]
parameters_mlp = {'activation': ['identity','logistic','tanh','relu'], 
                  'solver':['lbfgs','sgd','adam'], 
                  'learning_rate':['constant','invscaling','adaptive'],
                  'alpha': [0.01,0.001,0.0001]  
                  }
parameters.append(parameters_mlp)
parameters_knn = {"n_neighbors": [1,2,4,8,16,32], 
                 "weights": ['uniform','distance'],
                 "algorithm":['ball_tree', 'kd_tree', 'brute'],
                 "p":[1,2]
                  }
parameters.append(parameters_knn)
parameters_svm = {"kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
                 'decision_function_shape':('ovo', 'ovr'),
                 'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                 'C':[0.1,1,10,100,1000] 
                 }
parameters.append(parameters_svm)

# Classifiers Names

names = ['MLPClassifier',
        'KNeighborsClassifier',
        'SupportVectorClassifier',
        ]

In [ ]:
# Dataset Digits

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,test_size=0.20, shuffle=True)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Best Classifiers with GridSearchCV

path='/content/drive/MyDrive/RecPad/Models/'

def gridSearchCV(models,params,count):
    best_models=[]
    for i in tqdm(range(0,count)):
        model_grid = GridSearchCV(models[i], parameters[i], n_jobs=-1, verbose=1, cv=5)
        model_grid.fit(X_train,y_train)
        best_models.append(model_grid.best_estimator_)
    return best_models

best_model_list = gridSearchCV(classifiers,parameters,3)

In [ ]:
# Gerando o cabeçalho
log_accuracy = open('/content/drive/MyDrive/RecPad/Logs/accuracy.csv','a')
log_time = open('/content/drive/MyDrive/RecPad/Logs/time.log','a')


# Unique estimators
for x in range(len(best_model_list)):
  log_accuracy.write(str(names[x])+',')
  log_time.write(str(names[x])+',')


# Votting Hard
log_accuracy.write('VottingHard,')
log_time.write('VottingHard,')

# Votting Soft
log_accuracy.write('VottingSoft,')
log_time.write('VottingSoft,')

# Stacking
for x in range(len(best_model_list)):
  for y in range(len(best_model_list)):
    if x!=y:
      log_accuracy.write('{}{},'.format(names[x],names[y]))
      log_time.write('{}{},'.format(names[x],names[y]))

log_accuracy.write('\n')
log_time.write('\n')

log_accuracy.close()
log_time.close() 

In [ ]:
# Best Estimators log and print
log_file = open('/content/drive/MyDrive/RecPad/Logs/digits_best_estimators.log','a')
log_file.write("\nThe best estimators are:\n")
for i in range(len(classifiers)):
  log_file.write("\n")
  log_file.write(str(best_model_list[i]))
log_file.close() 

print(str(best_model_list[0])+'\n')
print(str(best_model_list[1])+'\n')
print(str(best_model_list[2])+'\n')

MLPClassifier(activation='logistic', alpha=0.01, batch_size=64, beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=400,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=42, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=1, warm_start=False)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=1, p=3,
                     weights='uniform')

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)



In [ ]:
# # Melhores estimadores

# # MLPClassifier
# best_model_list[0] = MLPClassifier(activation='logistic', alpha=0.01, batch_size=64, beta_1=0.9,
#               beta_2=0.999, early_stopping=True, epsilon=1e-08,
#               hidden_layer_sizes=(100,), learning_rate='constant',
#               learning_rate_init=0.001, max_fun=15000, max_iter=400,
#               momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
#               power_t=0.5, random_state=42, shuffle=True, solver='lbfgs',
#               tol=0.0001, validation_fraction=0.1, verbose=1, warm_start=False)

# # KNeighborsClassifier
# best_model_list[1] = KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
#                      metric_params=None, n_jobs=-1, n_neighbors=1, p=3,
#                      weights='uniform')

# # SupportVectorClassifier
# best_model_list[2] = SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovo', degree=3, gamma=0.1, kernel='rbf',
#     max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
#     verbose=False)

In [ ]:
# Experiment
from time import time

for i in range(30):
  digits = load_digits()
  X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.20, shuffle=True)

  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  log_accuracy = open('/content/drive/MyDrive/RecPad/Logs/accuracy.csv','a')
  log_time = open('/content/drive/MyDrive/RecPad/Logs/time.csv','a')


  # Unique estimators
  for x in range(len(best_model_list)):
    model = best_model_list[x]
    t0 = time()
    model.fit(X_train,y_train)
    t1 = (time() - t0)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred) * 100
    log_accuracy.write('{:.2f},'.format(accuracy))
    log_time.write('{:.2f},'.format(t1))


  # Voting Hard
  vot_clf = VotingClassifier(estimators=[('mlp', best_model_list[0]),
                                        ('knn', best_model_list[1]),
                                        ('svm', best_model_list[2])
                                        ], 
                            voting='hard',n_jobs=-1)
  t0 = time()
  vot_clf.fit(X_train,y_train)
  t1 = (time() - t0)
  y_pred = vot_clf.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred) * 100
  log_accuracy.write('{:.2f},'.format(accuracy))
  log_time.write('{:.2f},'.format(t1))

  # Votting Soft
  vot_clf = VotingClassifier(estimators=[('mlp', best_model_list[0]),
                                        ('knn', best_model_list[1]),
                                        ('svm', best_model_list[2])
                                        ], 
                            voting='soft',n_jobs=-1)
  t0 = time()
  vot_clf.fit(X_train,y_train)
  t1 = (time() - t0)
  y_pred = vot_clf.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred) * 100
  log_accuracy.write('{:.2f},'.format(accuracy))
  log_time.write('{:.2f},'.format(t1))

  # Stacking
  for x in range(len(best_model_list)):
    for y in range(len(best_model_list)):
      if x!=y:
        estimators = [(names[x],best_model_list[x])]
        stacking = StackingClassifier(estimators=estimators,final_estimator=best_model_list[y])
        t0 = time()
        stacking.fit(X_train, y_train)
        t1 = (time() - t0)
        y_pred = stacking.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred) * 100
        log_accuracy.write('{:.2f},'.format(accuracy))
        log_time.write('{:.2f},'.format(t1))

  log_accuracy.write('\n')
  log_time.write('\n')

  log_accuracy.close()
  log_time.close()  

In [ ]:
pd.read_csv('/content/drive/MyDrive/RecPad/Logs/accuracy.csv').shape

(30, 12)

In [ ]:
log_accuracy = '/content/drive/MyDrive/RecPad/Logs/accuracy.csv'
log_time = '/content/drive/MyDrive/RecPad/Logs/time.csv'

accuracy = pd.read_csv(log_accuracy)
time = pd.read_csv(log_time)

print(accuracy.mean(),'\n\n',time.mean())

In [ ]:
# Predição do melhor modelo

# clf.fit(X_train, y_train)
# predicted = clf.predict(X_test)

# _, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
# for ax, image, prediction in zip(axes, X_test, predicted):
#     ax.set_axis_off()
#     image = image.reshape(8, 8)
#     ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
#     ax.set_title(f'Prediction: {prediction}')

# Teste

In [ ]:
from time import time

best_model = SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
                decision_function_shape='ovo', degree=3, gamma=0.1, kernel='rbf',
                max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
                verbose=False)

estimator = [('SupportVectorClassifier',best_model)]

In [ ]:
# MLPClassifier

# Parâmetros
activation = ['identity','logistic','tanh','relu']
solver = ['lbfgs','sgd','adam']
learning_rate = ['constant','invscaling','adaptive']
alpha = [0.01,0.001,0.0001]  
digits = load_digits() # dataset
# Experimento 
rounds = 30
count = 0

for x in range(len(activation)): 
  for y in range(len(solver)):
    for z in range(len(learning_rate)):
      for w in range(len(alpha)):
        
        count = count + 1
        file =  '/content/drive/MyDrive/RecPad/Logs/SupportVectorClassifierMLPClassifier'+str(count)+'.csv'
        log = open(file,'a')
        log.write('{},{},{},{},{},{},{}\n'.format('Classifier','activation','solver','learning_rate','alpha','time','accuracy'))
        
        for i in range(rounds):
          
          X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.20, shuffle=True)
          scaler = MinMaxScaler()
          X_train = scaler.fit_transform(X_train)
          X_test = scaler.transform(X_test)

          model =  MLPClassifier(activation=activation[x], alpha=alpha[w], batch_size=64, beta_1=0.9,
                beta_2=0.999, early_stopping=True, epsilon=1e-08,
                hidden_layer_sizes=(100,), learning_rate=learning_rate[z],
                learning_rate_init=0.001, max_fun=15000, max_iter=400,
                momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
                power_t=0.5, random_state=42, shuffle=True, solver=solver[y],
                tol=0.0001, validation_fraction=0.1, verbose=False, warm_start=False)

          stacking = StackingClassifier(estimators=estimator,final_estimator=model)
          stacking.fit(X_train,y_train)
          t0 = time()
          stacking.fit(X_train, y_train)
          t1 = (time() - t0)
          y_pred = stacking.predict(X_test)
          accuracy = accuracy_score(y_test, y_pred) * 100
          log.write('{},{},{},{},{:.4f},{:.2f},{:.2f}\n'.format('SupportVectorClassifierMLPClassifier',activation[x],solver[y],learning_rate[z],alpha[w],t1,accuracy))
        log.close()

In [ ]:
logs = open('/content/drive/MyDrive/RecPad/SupportVectorClassifierMLPClassifier.csv','a')
for i in range(1,109):
  log = '/content/drive/MyDrive/RecPad/Logs/SupportVectorClassifierMLPClassifier'+str(i)+'.csv'
  df = pd.read_csv(log)
  df.reset_index(inplace=True)
  # print('{},{},{},{},{},{},{:.2f}%,{}s' .format(df.iloc[0,1], df.iloc[0,2], df.iloc[0,3], df.iloc[0,4],df.iloc[0,5],df.iloc[0,6], df['accuracy'].mean(), df['time'].mean()))
  logs.write('{},{},{},{},{},{},{:.2f}%,{}s\n' .format(df.iloc[0,1], df.iloc[0,2], df.iloc[0,3], df.iloc[0,4],df.iloc[0,5],df.iloc[0,6], df['accuracy'].mean(), df['time'].mean()))
logs.close()

In [ ]:
# KNeighborsClassifier

# Parâmetros
n_neighbors = [1,2,4,8,16,32] 
weights = ['uniform','distance']
algorithm = ['ball_tree', 'kd_tree', 'brute']
p = [1,2,3]
digits = load_digits() # dataset

# Experimento 
rounds = 30
count = 0

for x in range(len(n_neighbors)): 
  for y in range(len(weights)):
    for z in range(len(algorithm)):
      for w in range(len(p)):
       
        count = count + 1
        file =  '/content/drive/MyDrive/RecPad/Logs/SupportVectorClassifierKNeighborsClassifier'+str(count)+'.csv'
        log = open(file,'a')
        log.write('{},{},{},{},{},{},{}\n'.format('Classifier','n_neighbors','weights','algorithm','p','time','accuracy'))
     
        for i in range(rounds):

          X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.20, shuffle=True)
          scaler = MinMaxScaler()
          X_train = scaler.fit_transform(X_train)
          X_test = scaler.transform(X_test)

          model = KNeighborsClassifier(algorithm=algorithm[z], leaf_size=30, metric='minkowski',
                                       metric_params=None, n_jobs=-1, n_neighbors=n_neighbors[x], p=p[w],
                                       weights=weights[y])

          stacking = StackingClassifier(estimators=estimator,final_estimator=model)
          stacking.fit(X_train,y_train)
          t0 = time()
          stacking.fit(X_train, y_train)
          t1 = (time() - t0)
          y_pred = stacking.predict(X_test)
          accuracy = accuracy_score(y_test, y_pred) * 100
          log.write('{},{},{},{},{},{:.2f},{:.2f}\n'.format('SupportVectorClassifierKNeighborsClassifier',n_neighbors[x],weights[y],algorithm[z],p[w],t1,accuracy))
        log.close()

In [ ]:
logs = open('/content/drive/MyDrive/RecPad/SupportVectorClassifierKNeighborsClassifier.csv','a')
for i in range(1,109):
  log = '/content/drive/MyDrive/RecPad/Logs/SupportVectorClassifierKNeighborsClassifier'+str(i)+'.csv'
  df = pd.read_csv(log)
  df.reset_index(inplace=True)
  # print('{},{},{},{},{},{},{:.2f}%,{}s' .format(df.iloc[0,1], df.iloc[0,2], df.iloc[0,3], df.iloc[0,4],df.iloc[0,5],df.iloc[0,6], df['accuracy'].mean(), df['time'].mean()))
  logs.write('{},{},{},{},{},{},{:.2f}%,{}s\n' .format(df.iloc[0,1], df.iloc[0,2], df.iloc[0,3], df.iloc[0,4],df.iloc[0,5],df.iloc[0,6], df['accuracy'].mean(), df['time'].mean()))
logs.close()

In [ ]:
import scipy.stats as stats

log_SupportVectorClassifier = pd.read_csv('/content/drive/MyDrive/RecPad/SupportVectorClassifier.csv','a')
log_Ensemble = pd.read_csv('/content/drive/MyDrive/RecPad/VotingHard.csv','a')

SupportVectorClassifier = log_SupportVectorClassifier['accuracy']
Ensemble = log_Ensemble['accuracy']

stats.wilcoxon(Best_Estimator, Ensemble)